In [1]:
# -*-coding:utf-8
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec=TfidfVectorizer()

#创建文档列表 documents
documents=[
    'this is the bayes document',
    'this is the second document',
    'and the third one',
    'is this the document'
]
# 使用 tfidf_vec 对 documents 进行拟合，得到 TF-IDF 矩阵
tfidf_matrix = tfidf_vec.fit_transform(documents)

print('不重复的词：',tfidf_vec.get_feature_names())
print('每个单词的 ID：',tfidf_vec.vocabulary_)
print('每个单词的 tfidf 值：',tfidf_matrix.toarray())

不重复的词： ['and', 'bayes', 'document', 'is', 'one', 'second', 'the', 'third', 'this']
每个单词的 ID： {'this': 8, 'is': 3, 'the': 6, 'bayes': 1, 'document': 2, 'second': 5, 'and': 0, 'third': 7, 'one': 4}
每个单词的 tfidf 值： [[0.         0.63314609 0.40412895 0.40412895 0.         0.
  0.33040189 0.         0.40412895]
 [0.         0.         0.40412895 0.40412895 0.         0.63314609
  0.33040189 0.         0.40412895]
 [0.55280532 0.         0.         0.         0.55280532 0.
  0.28847675 0.55280532 0.        ]
 [0.         0.         0.52210862 0.52210862 0.         0.
  0.42685801 0.         0.52210862]]


In [7]:
import jieba
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

加载停用词

In [8]:
# with open('/Users/wangyunfei/Desktop/text_classification-master/text classification/stop/stopword.txt','r') as f:
#     stop_words=[line.strip() for line in f.readlines()]
stop_words = open('./text_classification-master/text classification/stop/stopword.txt', 'r', encoding='utf-8').read()
stop_words = stop_words.encode('utf-8').decode('utf-8-sig') # 列表头部\ufeff处理
stop_words = stop_words.split('\n') # 根据分隔符分隔
# print(stop_words)

对文本分词

In [ ]:
def cut_words(file_path):
    '''
    对文本进行分词
    :return: 用空格进行分词的字符串
    '''
    texit_with_space=''
    text=open(file_path,'r',encoding='gb18030').read()
    text_cut=jieba.cut(text)
    for word in text_cut:
        texit_with_space += word + ''
    return texit_with_space

def load_file(file_dir,label):
    '''
    将路径下的所有文件加载
    :return: 分词后的文档列表和标签
    '''
    file_list = os.listdir(file_dir)
    words_list = []
    lables_list = []
    for file in file_list:
        file_path = file_dir+'/'+file
        words_list.append(cut_words(file_path))
        lables_list.append(label)
    return words_list,lables_list

训练数据

In [9]:
train_words_list1,train_labels1=load_file('./text_classification-master/text classification/train/女性','女性')
train_words_list2,train_labels2=load_file('./text_classification-master/text classification/train/体育','体育')
train_words_list3,train_labels3=load_file('./text_classification-master/text classification/train/文学','文学')
train_words_list4,train_labels4=load_file('./text_classification-master/text classification/train/校园','校园')

train_words_list=train_words_list1+train_words_list2+train_words_list3+train_words_list4
train_labels = train_labels1+train_labels2+train_labels3+train_labels4

测试数据

In [ ]:
test_words_list1,test_labels1=load_file('./text_classification-master/text classification/test/女性','女性')
test_words_list2,test_labels2=load_file('./text_classification-master/text classification/test/体育','体育')
test_words_list3,test_labels3=load_file('./text_classification-master/text classification/test/文学','文学')
test_words_list4,test_labels4=load_file('./text_classification-master/text classification/test/校园','校园')

test_words_list=test_words_list1+test_words_list2+test_words_list3+test_words_list4
test_labels = test_labels1+test_labels2+test_labels3+test_labels4

计算单词权重，直接创建 TfidfVectorizer 类，然后使用 fit_transform 进行拟合，得到 TF-IDF 特征空间 features，你可以理解为选出来的分词就是特征。我们计算这些特征在文档上的特征向量，得到特征空间 features。        
max_df 参数用来描述单词在文档中的最高出现率，假设 max_df =0.5，代表单词在 50% 的文档中都出现过，那么它只携带了非常少的信息，因此就不作为分词统计。

In [12]:
tf = TfidfVectorizer(stop_words=stop_words,max_df=0.5)

train_features=tf.fit_transform(train_words_list)
test_features=tf.transform(test_words_list)

多项式贝叶斯分类器， alpha为平滑参数，假如一个单词在训练样本中没有出现，这个单词的概率就会被计算为0。但训练集样本只是整体的抽样情况，我们不能因为一个事件没有观察到，就认为整个事件的概率为 0。了解了这个问题，我们需要做平滑处理。
当 alpha = 1，就是采用 Laplace 平滑，Laplace 平滑就是采用加 1 的方式，来统计没有出现过的单词的概率。这样当训练样本很大的时候，加 1 得到的概率变化可以忽略不计，也同时避免了零概率问题。            
当 0<alpha<1，采用的是 Lidstone 平滑，对于 Lidstone 平滑来说，alpha 越小，迭代次数

In [13]:
clf = MultinomialNB(alpha=0.001).fit(train_features,train_labels)

predicted_labels = clf.predict(test_features)

# 计算准确率
print("准确率为：",metrics.accuracy_score(test_labels,predicted_labels))

准确率为： 0.765
